# Asset Analysis
Developed by Steve Kuhlman

In [5]:
using WGLMakie, AlgebraOfGraphics,  XLSX, DataFrames, Dates

In [6]:
#TODO Move functions to a file
function readMarket(marketFile)
  mdf=DataFrame(XLSX.readtable(marketFile, 1)...);
  mdf.Close=convert(Vector{Float64}, mdf.Close);    
  mdf
end

stripXLSX(s)=split(s, ".")[1]

function readMarkets(dataDir)
  marketFiles=readdir(dataDir)    
  marketPaths=map(x -> joinpath(dataDir, x), marketFiles)
  markets = map(readMarket, marketPaths) 
  marketArr = map(readMarket,  marketPaths)
  markets=reduce(vcat, marketArr)
  marketNames=map(stripXLSX, marketFiles)
  (markets, marketNames)
end

readMarkets (generic function with 1 method)

In [8]:
#Note: Enter the folder containing data here
dataDir="../data/market"
(markets, marketNames)=readMarkets(dataDir);

In [9]:
"Calculate the minumum closing value for each equity"
function getMinClose(gmarkets)
  closeHL=combine(gmarkets, :Close => minimum)
  minClose=Dict{String, Float64}()
  [minClose[closeHL[r, :Ticker]] = closeHL[r, :Close_minimum]  for r in 1:size(closeHL, 1)]
  minClose
end

"Get the first and last date for the charts"
function getDateRange(market)
  #TODO get actual first and last, don't assume positions
  first=market[1,:timestamp]
  last=market[end,:timestamp]
  dateRange=first : Day(1) : last
end

function plotTicker(markets, marketNames, pos)
  dateRange=getDateRange(markets[pos])
  lines!(days, markets[pos].Close, label=marketNames[pos])
end

plotTicker (generic function with 1 method)

In [10]:
dateRange=getDateRange(markets)
gmarkets=groupby(markets, :Ticker);
minClose=getMinClose(gmarkets)
normalize(x,y)= (x / minClose[y], minClose[y])
transform!(markets, [:Close, :Ticker] => ByRow(normalize) => [:CloseNorm, :minClose]);


In [11]:
"Plot normalized values of investments"
plt=data(markets) * mapping(:timestamp, :CloseNorm, color=:Ticker) * visual(Lines)
draw(plt)


(process:24764): GLib-GIO-WARNING **: 15:48:50.420: Unexpectedly, UWP app `6760NGPDFLab.PDFX_1.3.41.0_x64__sbe4t8mqwq93a' (AUMId `6760NGPDFLab.PDFX_sbe4t8mqwq93a!App') supports 32 extensions but has no verbs


In [ ]:
"Plot market prices of investments"
#plt=data(markets) * mapping(:timestamp, :Close, color=:Ticker) * visual(Lines)
#draw(plt)